<a href="https://colab.research.google.com/github/vaniamv/dataprocessing/blob/main/spark_streaming/challenges/final_challenges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up PySpark

In [ ]:
%pip install pyspark

# Context
Message events are coming from platform message broker (kafka, pubsub, kinesis...).
You need to process the data according to the requirements.

Message schema:
- timestamp
- value
- event_type
- message_id
- country_id
- user_id



# Challenge 1

Step 1
- Change exising producer
	- Change parquet location to "/content/lake/bronze/messages/data"
	- Add checkpoint (/content/lake/bronze/messages/checkpoint)
	- Delete /content/lake/bronze/messages and reprocess data
	- For reprocessing, run the streaming for at least 1 minute, then stop it

Step 2
- Implement new stream job to read from messages in bronze layer and split result in two locations
	- "messages_corrupted"
		- logic: event_status is null, empty or equal to "NONE"
    - extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages_corrupted/data

	- "messages"
		- logic: not corrupted data
		- extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages/data

	- technical requirements
		- add checkpint (choose location)
		- use StructSchema
		- Set trigger interval to 5 seconds
		- run streaming for at least 20 seconds, then stop it

	- alternatives
		- implementing single streaming job with foreach/- foreachBatch logic to write into two locations
		- implementing two streaming jobs, one for messages and another for messages_corrupted
		- (paying attention on the paths and checkpoints)


  - Check results:
    - results from messages in bronze layer should match with the sum of messages+messages_corrupted in the silver layer

In [2]:
%pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.7 MB/s eta 0:00:00


# Producer

In [54]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from faker import Faker
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Test streaming').getOrCreate()
sc = spark.sparkContext

fake = Faker()
messages = [fake.uuid4() for _ in range(50)]

def enrich_data(df, messages=messages):
  fake = Faker()
  new_columns = {
      'event_type': F.lit(fake.random_element(elements=('OPEN', 'RECEIVED', 'SENT', 'CREATED', 'CLICKED', '', 'NONE'))),
      'message_id': F.lit(fake.random_element(elements=messages)),
      'channel': F.lit(fake.random_element(elements=('CHAT', 'EMAIL', 'SMS', 'PUSH', 'OTHER'))),
      'country_id': F.lit(fake.random_int(min=2000, max=2015)),
      'user_id': F.lit(fake.random_int(min=1000, max=1050)),
  }
  df = df.withColumns(new_columns)
  return df

def insert_messages(df: DataFrame, batch_id):
  enrich = enrich_data(df)
  enrich.write.mode("append").format("parquet").save("content/lake/bronze/messages/data")

# read stream
df_stream = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# write stream
query = (df_stream.writeStream
.outputMode('append')
.option('checkpointLocation', 'content/lake/bronze/checkpoint')
.trigger(processingTime='1 seconds')
.foreachBatch(insert_messages)
.start()
)

query.awaitTermination(60)


False

In [59]:
query.isActive

True

In [46]:
query.stop()

In [19]:
!rm -rf content/lake/bronze/messages

In [15]:
df = spark.read.format("parquet").load("content/lake/bronze/messages/*")
df.show()

+--------------------+-----+----------+--------------------+-------+----------+-------+
|           timestamp|value|event_type|          message_id|channel|country_id|user_id|
+--------------------+-----+----------+--------------------+-------+----------+-------+
|2024-12-05 16:35:...|    5|   CREATED|6883a035-5083-4f1...|  OTHER|      2014|   1010|
|2024-12-05 16:35:...|    7|   CREATED|6883a035-5083-4f1...|  OTHER|      2014|   1010|
|2024-12-05 16:35:...|    6|   CREATED|6883a035-5083-4f1...|  OTHER|      2014|   1010|
|2024-12-05 16:35:...|    8|   CREATED|6883a035-5083-4f1...|  OTHER|      2014|   1010|
|2024-12-05 16:35:...|   59|   CLICKED|51fe7589-75d6-445...|    SMS|      2015|   1046|
|2024-12-05 16:36:...|   61|   CLICKED|51fe7589-75d6-445...|    SMS|      2015|   1046|
|2024-12-05 16:34:...|    0|      NONE|44b123ce-a0fb-407...|    SMS|      2007|   1004|
|2024-12-05 16:35:...|    2|      NONE|44b123ce-a0fb-407...|    SMS|      2007|   1004|
|2024-12-05 16:35:...|    4|    

In [31]:
df = spark.read.format("parquet").load("content/lake/bronze/messages/*")
df.show()

+--------------------+-----+----------+--------------------+-------+----------+-------+
|           timestamp|value|event_type|          message_id|channel|country_id|user_id|
+--------------------+-----+----------+--------------------+-------+----------+-------+
|2024-12-05 17:00:...|   67|          |76c3c11d-81a2-4af...|    SMS|      2003|   1015|
|2024-12-05 17:00:...|   69|          |76c3c11d-81a2-4af...|    SMS|      2003|   1015|
|2024-12-05 17:00:...|   71|          |76c3c11d-81a2-4af...|    SMS|      2003|   1015|
|2024-12-05 17:00:...|   73|          |76c3c11d-81a2-4af...|    SMS|      2003|   1015|
|2024-12-05 17:00:...|   75|          |76c3c11d-81a2-4af...|    SMS|      2003|   1015|
|2024-12-05 17:01:...|   77|          |76c3c11d-81a2-4af...|    SMS|      2003|   1015|
|2024-12-05 17:01:...|   79|          |76c3c11d-81a2-4af...|    SMS|      2003|   1015|
|2024-12-05 17:01:...|   81|          |76c3c11d-81a2-4af...|    SMS|      2003|   1015|
|2024-12-05 17:01:...|   83|    

In [61]:
df.count()

189

# Additional datasets

In [14]:
countries = [
    {"country_id": 2000, "country": "Brazil"},
    {"country_id": 2001, "country": "Portugal"},
    {"country_id": 2002, "country": "Spain"},
    {"country_id": 2003, "country": "Germany"},
    {"country_id": 2004, "country": "France"},
    {"country_id": 2005, "country": "Italy"},
    {"country_id": 2006, "country": "United Kingdom"},
    {"country_id": 2007, "country": "United States"},
    {"country_id": 2008, "country": "Canada"},
    {"country_id": 2009, "country": "Australia"},
    {"country_id": 2010, "country": "Japan"},
    {"country_id": 2011, "country": "China"},
    {"country_id": 2012, "country": "India"},
    {"country_id": 2013, "country": "South Korea"},
    {"country_id": 2014, "country": "Russia"},
    {"country_id": 2015, "country": "Argentina"}
]

countries = spark.createDataFrame(countries)

# Streaming Messages x Messages Corrupted

In [34]:
df.select('event_type').distinct().collect()


[Row(event_type='CLICKED'),
 Row(event_type='CREATED'),
 Row(event_type='RECEIVED'),
 Row(event_type='NONE'),
 Row(event_type=''),
 Row(event_type='OPEN'),
 Row(event_type='SENT')]

In [25]:
# TODO
from pyspark.sql.functions import col, lit

corrupted_condition = (col("event_type").isNull() |
                       (col("event_type") == "") |
                       (col("event_type") == "NONE"))

In [48]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType


In [62]:
def insert_messages_silver(df: DataFrame, batch_id):
  corrupted_df = df.filter(corrupted_condition)
  corrupted_df.write.mode("append").format("parquet").save("content/lake/silver/messages_corrupted/data")

  non_corrupted_df = df.filter(~corrupted_condition)
  non_corrupted_df.write.mode("append").format("parquet").save("content/lake/silver/messages/data")

schema = StructType([
    StructField("timestamp", TimestampType(), True),
    StructField("event_type", StringType(), True),
    # Add other fields and their types as needed
])

# Read the streaming data with the specified schema
messages_df = spark.readStream.format("parquet").schema(schema).load("content/lake/bronze/messages/data")

#messages_df = spark.readStream.format("parquet").load("content/lake/bronze/messages/data")

query_1 = (messages_df.writeStream
.outputMode('append')
.option('checkpointLocation', 'content/lake/silver/checkpoint')
.trigger(processingTime='1 seconds')
.foreachBatch(insert_messages_silver)
.start()
)


In [65]:
df = spark.read.format("parquet").load("content/lake/silver/messages/*")
df.show()

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/content/content/lake/silver/messages/*.

## Checking data

In [ ]:
# TODO

# Challenge 2

- Run business report
- But first, there is a bug in the system which is causing some duplicated messages, we need to exclude these lines from the report

- removing duplicates logic:
  - Identify possible duplicates on message_id, event_type and channel
  - in case of duplicates, consider only the first message (occurrence by timestamp)
  - Ex:
    In table below, the correct message to consider is the second line

```
    message_id | channel | event_type | timestamp
    123        | CHAT    | CREATED    | 10:10:01
    123        | CHAT    | CREATED    | 07:56:45 (first occurrence)
    123        | CHAT    | CREATED    | 08:13:33
```

- After cleaning the data we're able to create the busines report

In [ ]:
# dedup data
from pyspark.sql import functions as F
from pyspark.sql.window import Window
df = spark.read.format("parquet").load("content/lake/silver/messages")
dedup = df.withColumn("row_number", F.row_number().over(Window.partitionBy("message_id", "event_type", "channel").orderBy("timestamp"))).filter("row_number = 1").drop("row_number")

### Report 1
  - Aggregate data by date, event_type and channel
  - Count number of messages
  - pivot event_type from rows into columns
  - schema expected:
  
```
|      date|channel|CLICKED|CREATED|OPEN|RECEIVED|SENT|
+----------+-------+-------+-------+----+--------+----+
|2024-12-03|    SMS|      4|      4|   1|       1|   5|
|2024-12-03|   CHAT|      3|      7|   5|       8|   4|
|2024-12-03|   PUSH|   NULL|      3|   4|       3|   4|
```

In [ ]:
# report 1
# TODO

## Report 2

- Identify the most active users by channel (sorted by number of iterations)
- schema expected:

```
+-------+----------+----+-----+-----+----+---+
|user_id|iterations|CHAT|EMAIL|OTHER|PUSH|SMS|
+-------+----------+----+-----+-----+----+---+
|   1022|         5|   2|    0|    1|   0|  2|
|   1004|         4|   1|    1|    1|   1|  0|
|   1013|         4|   0|    0|    2|   1|  1|
|   1020|         4|   2|    0|    1|   1|  0|
```


In [ ]:
# report 2
# TODO

# Challenge 3

In [ ]:
# Theoretical question:

# A new usecase requires the message data to be aggregate in near real time
# They want to build a dashboard embedded in the platform website to analyze message data in low latency (few minutes)
# This application will access directly the data aggregated by streaming process

# Q1:
- What would be your suggestion to achieve that using Spark Structure Streaming?
Or would you choose a different data processing tool?

- Which storage would you use and why? (database?, data lake?, kafka?)

